In [ ]:
from data import generer
import pandas as pd
import duckdb
import pathlib
import shutil
from loguru import logger

## Parquet

### Lage parquet-filer

In [ ]:
mappe = pathlib.Path("data")
out_path = mappe / "data.parquet"

try:
    shutil.rmtree(mappe)
except FileNotFoundError as e:
    logger.info(e)
mappe.mkdir(exist_ok=True, parents=True)

In [ ]:
n = int(1e6)
df = generer(n)
df.head()

In [ ]:
df.dtypes

In [ ]:
df.tidspunkt.min(), df.tidspunkt.max()

In [ ]:
df.to_parquet(out_path, engine='pyarrow')
df.to_csv(mappe / "data.csv", index=False)

In [ ]:
!ls -lh data

In [ ]:
df = pd.read_parquet(out_path, engine="pyarrow")#, dtype_backend="pyarrow")

In [ ]:
df.head()

In [ ]:
df.memory_usage(deep=True).sum() // 1e6

In [ ]:
df.dtypes

In [ ]:
df = pd.read_csv(mappe / 'data.csv')

In [ ]:
df.memory_usage(deep=True).sum() / 1e6

### Hent litt data fra en parquet-fil

In [ ]:
in_path = mappe / "data.parquet"
pd.read_parquet(in_path).query("kategori == 'vedtak'")[['tidspunkt']].head()

In [ ]:
pd.read_parquet(in_path, columns=['tidspunkt'], filters=[('kategori', '=', 'vedtak')])

### Hente litt data fra flere parquet-filer

In [ ]:
!cp data/data.parquet data/data2.parquet
!cp data/data.parquet data/data3.parquet

In [ ]:
!ls -lah data

In [ ]:
paths = mappe.glob('*.parquet')
# for path in g:
#     print(path)

In [ ]:
dfs = (pd.read_parquet(path=path, engine='pyarrow') for path in paths)
# for df in gdf:
#     print(len(df))

In [ ]:
pd.concat(dfs)

## DuckDB

### Les parquet-filer

In [ ]:
!ls -l data/data*.parquet

In [ ]:
duckdb.query("""--sql
select id, tidspunkt as antall from 'data/data*.parquet'
where tidspunkt between '2023-03-01 01:00' and '2023-03-01 02:00'
and kategori = 'vedtak'
""").to_df()

### Les mange parquet-filer

In [ ]:
!./kopier.zsh

In [ ]:
!du -hs data

In [ ]:
paths = mappe.glob('data_*.parquet')
dfs = (pd.read_parquet(path) for path in paths)
# sum((1 for _ in paths))

In [ ]:
antall_rader = 0
from tqdm.auto import tqdm
for df in tqdm(dfs, total=1000):
    antall_rader += len(df)
antall_rader

In [ ]:
# duckdb.query("""--sql
# COPY (SELECT *, tidspunkt::date dato FROM 'data/data*.parquet')
# TO 'data/partisjonert'
# (FORMAT PARQUET, PARTITION_BY (dato));
# """)

In [ ]:
df = duckdb.query("""--sql
select * from 'data/data*.parquet'
where kategori in ('søknad', 'vedtak')
and tidspunkt::date = '2023-03-01'
order by tidspunkt desc
""").to_df()

In [ ]:

# for kategori in ['inntektsmelding', 'vedtak', 'søknad', 'sykmelding']:
#     con.query(f"""--sql
#     CREATE TABLE {kategori} AS
#     SELECT * FROM 'data/data.parquet'
#     WHERE kategori = '{kategori}'
#     """)
# con.query("select * from inntektsmelding limit 1")

### Transformer parquet-filer

In [ ]:
in_path = mappe / "data*.parquet"
out_path = mappe / "daglig.parquet"
out_path.unlink(missing_ok=True)

In [ ]:
duckdb.query(f"""--sql
COPY (
select
    tidspunkt::date dato,
    kategori,
    count(1) antall,
from '{in_path}'
where tidspunkt between '2023-03-01' and '2023-04-01'
group by kategori, dato
order by dato
)
TO '{out_path}'
""")

In [ ]:
df = pd.read_parquet(out_path)
df

In [ ]:
duckdb.query(f"""--sql
SELECT * FROM '{out_path}'
""")